In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import cv2
from math import sqrt, pow

model = cv2.dnn.readNetFromCaffe("/content/drive/My Drive/Coding/SSD_MobileNet_prototxt.txt",
                                   "/content/drive/My Drive/Coding/SSD_MobileNet.caffemodel")
cam = cv2.VideoCapture("/content/drive/My Drive/Coding/dvpm2.avi")
F=615
res=(int(cam.get(3)),int(cam.get(4)))
w=int(cam.get(3))
h=int(cam.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('dvpm_result.avi', fourcc, 20.0, res)
frameno=0

while True:
  (rval, im) = cam.read()
  print(frameno)
  frameno=frameno+1
  # predicting 
  if rval == True:
    #im=cv2.resize(im, (299, 299))

    resized=cv2.resize(im, (300, 300))
    blob = cv2.dnn.blobFromImage(resized, 0.008, (300, 300), 127.5)
    model.setInput(blob)
    det=model.forward()
    coords_cm={}
    coords={}
    tooclose=[]
    for i in range(det.shape[2]):
      if det[0, 0, i, 2]>0.2: #probability of detection being detected object
        if int(det[0,0,i,1]==15): #is a human
          coords_x=det[0, 0, i, 3:7]
          coords_x=coords_x*np.array([w, h, w, h])
          (x,y,x2,y2)=coords_x.astype("int")
          coords[i]=(x,y,x2,y2)
          scalefactor=165/round(y2-y,2)
          coords_cm[i]=(round((x+x2)/2,2)*scalefactor, round((y+y2)/2,2)*scalefactor, F*scalefactor)
    
    for i in coords_cm.keys():
      for j in coords_cm.keys():
        if i<j:
          d=sqrt(pow(coords_cm[i][0]-coords_cm[j][0],2) + pow(coords_cm[i][1]-coords_cm[j][1],2) + pow(coords_cm[i][2]-coords_cm[j][2],2))
          #print(i,"to",j,"is",d,"cm")
          if d < 150:
            tooclose.append(i)
            tooclose.append(j)
    
    #time to plot rectangles!
    for i in coords.keys():
      (xa, ya, xb, yb)=coords[i]
      if i in tooclose:
        color=(0,0,255)
      else:
        color=(0,255,0)
      cv2.rectangle(im, (xa,ya), (xb,yb), color, 2)
    
    out.write(im)

  else:
    print("VIDEO FINISHED")
    break
  if cv2.waitKey(1) & 0xFF == 27: #Esc
    break

cam.release()
out.release()